In [42]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import sys
sys.path.append('..')

from helper import nn
from helper import logistic_regression as lr

In [86]:
raw_X, raw_y = nn.load_data()
print(raw_X.shape)
print(raw_y.shape)

(5000, 400)
(5000,)


# prepare data

In [31]:
# add intercept=1 for x0
X = np.array([np.concatenate([np.ones(1), x]) for x in raw_X])
X.shape

(5000, 401)

In [32]:
# y have 10 categories here. 1..10, they represent digit 0 as category 10 because matlab index start at 1
# I'll ditit 0, index 0 again
y_matrix = []

for k in range(1, 11):
    y_matrix.append((raw_y == k).astype(int))

# last one is k==10, it's digit 0, bring it to the first position
y_matrix = [y_matrix[-1]] + y_matrix[:-1]
y = np.array(y_matrix)

y.shape

(10, 5000)

# train 1 model

In [35]:
t0 = lr.logistic_regression(X, y[0])

In [40]:
print(t0.shape)
y_pred = lr.predict(X, t0)
print('Accuracy={}'.format(np.mean(y[0] == y_pred)))

(401,)
Accuracy=0.9974


Is this real......

# train k model
this will take 20 mins or so...

%%timeit
k_theta = np.array([lr.logistic_regression(X, y[k]) for k in range(10)])
print(k_theta.shape)

with open('k_theta.pickle', 'wb') as f:
    pickle.dump(k_theta, f)

In [92]:
with open('k_theta.pickle', 'rb') as f:
    k_theta = pickle.load(f)

k_theta.shape

(10, 401)

# making prediction
* think about the shape of k_theta, now you are making $X\times\theta^T$
> $(5000, 401) \times (10, 401).T = (5000, 10)$
* after that, you run sigmoid to get probabilities and for each row, you find the highest prob as the answer

In [93]:
prob_matrix = lr.sigmoid(X @ k_theta.T)

In [94]:
np.set_printoptions(suppress=True)
prob_matrix

array([[ 0.99577077,  0.        ,  0.00053511, ...,  0.00006463,
         0.0000393 ,  0.00172808],
       [ 0.99834671,  0.0000001 ,  0.00005613, ...,  0.00009676,
         0.00000291,  0.00008511],
       [ 0.99139059,  0.        ,  0.00056846, ...,  0.00000655,
         0.02654266,  0.00197298],
       ..., 
       [ 0.00000068,  0.04133786,  0.00320778, ...,  0.00012693,
         0.00298173,  0.70774295],
       [ 0.00001848,  0.00000013,  0.00000009, ...,  0.00164798,
         0.06804472,  0.86106229],
       [ 0.02874759,  0.        ,  0.00012969, ...,  0.36610147,
         0.00500491,  0.14817185]])

In [95]:
y_pred = np.argmax(prob_matrix, axis=1)

In [96]:
y_answer = raw_y.copy()
y_answer[y_answer==10] = 0

In [97]:
np.mean(y_pred == y_answer)

0.9446